In [2181]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [2182]:
df = pd.read_csv("train.csv")

/tmp/ipython-input-2292805398.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


# Preprocessing

In [2183]:
df.shape

(100000, 28)

In [2184]:
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [2185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [2186]:
df = df.drop(columns=['ID', 'Month', 'Name', 'SSN'])

In [2228]:
clos_to_covert = ['Annual_Income', 'Monthly_Inhand_Salary', 'Outstanding_Debt',
                'Credit_Utilization_Ratio', 'Total_EMI_per_month', 'Amount_invested_monthly',
                'Changed_Credit_Limit', 'Monthly_Balance', "Num_of_Loan", "Num_of_Delayed_Payment"]

for col in clos_to_covert:
        df[col] = df[col].astype(str).str.replace(r"[^0-9.]", "", regex=True)
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [2188]:
df["Age"] = df["Age"].astype(str).str.replace(r"[^0-9]", "", regex=True)
df["Age"] = pd.to_numeric(df["Age"], errors="coerce")

In [2189]:
median_age = df.loc[(df["Age"] >= 18) & (df["Age"] <= 90), "Age"].median()
df.loc[(df["Age"] < 18) | (df["Age"] > 90), "Age"] = median_age

In [2190]:
df["Age"].unique()

array([23, 34, 28, 54, 55, 21, 31, 33, 30, 24, 44, 45, 40, 41, 32, 35, 36,
       39, 37, 20, 46, 26, 42, 19, 48, 38, 43, 22, 18, 27, 25, 47, 53, 56,
       29, 49, 51, 50, 52])

In [2191]:
df["Occupation"].unique()

array(['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur',
       'Developer', 'Lawyer', 'Media_Manager', 'Doctor', 'Journalist',
       'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer',
       'Architect'], dtype=object)

In [2192]:
df["Occupation"] = df["Occupation"].replace(["_______"], np.nan)

In [2193]:
df["Occupation"].isna().sum()

np.int64(7062)

In [2194]:
df["Occupation"] = df["Occupation"].fillna(df["Occupation"].mode()[0])

In [2195]:
df["Occupation"].isna().sum()

np.int64(0)

In [2196]:
freq_map = df["Occupation"].value_counts().to_dict()
df["Occupation"] = df["Occupation"].map(freq_map)

In [2197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Age                       100000 non-null  int64  
 2   Occupation                100000 non-null  int64  
 3   Annual_Income             100000 non-null  float64
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  int64  
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   object 
 12  Changed_Credit_Limit      97909 non-null   float64
 13  Num_Credit_Inquiries      98035 non-null   fl

In [2198]:
df["Annual_Income"].isna().sum()

np.int64(0)

In [2199]:
df["Annual_Income"] = df.groupby("Customer_ID")["Annual_Income"].transform(lambda x: x.fillna(x.median()))
df["Annual_Income"] = df["Annual_Income"].fillna(df["Annual_Income"].median())

In [2200]:
df["Annual_Income"].isna().sum()

np.int64(0)

In [2201]:
df["Monthly_Inhand_Salary"].isna().sum()


np.int64(15002)

In [2202]:
df["Monthly_Inhand_Salary"] = df.groupby("Customer_ID")["Monthly_Inhand_Salary"].transform(lambda x: x.fillna(x.median()))
df["Monthly_Inhand_Salary"] = df["Monthly_Inhand_Salary"].fillna(df["Monthly_Inhand_Salary"].median())

In [2203]:
df["Monthly_Inhand_Salary"].isna().sum()

np.int64(0)

In [2204]:
def convert_credit_history(x):
    try:
        parts = x.split()
        years = int(parts[0])
        months = int(parts[3])
        return years * 12 + months
    except:
        return np.nan

df["Credit_History_Age_Months"] = df["Credit_History_Age"].apply(convert_credit_history)
df.drop('Credit_History_Age', axis=1, inplace=True)

In [2205]:
df['Credit_History_Age_Months'].isna().sum()

np.int64(9030)

In [2206]:
df["Credit_History_Age_Months"] = df["Credit_History_Age_Months"].fillna(df["Credit_History_Age_Months"].median())
df["Credit_History_Age_Months"] = df["Credit_History_Age_Months"].astype(int)

In [2207]:
df['Credit_History_Age_Months'].isna().sum()

np.int64(0)

In [2208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Customer_ID                100000 non-null  object 
 1   Age                        100000 non-null  int64  
 2   Occupation                 100000 non-null  int64  
 3   Annual_Income              100000 non-null  float64
 4   Monthly_Inhand_Salary      100000 non-null  float64
 5   Num_Bank_Accounts          100000 non-null  int64  
 6   Num_Credit_Card            100000 non-null  int64  
 7   Interest_Rate              100000 non-null  int64  
 8   Num_of_Loan                100000 non-null  int64  
 9   Type_of_Loan               88592 non-null   object 
 10  Delay_from_due_date        100000 non-null  int64  
 11  Num_of_Delayed_Payment     92998 non-null   object 
 12  Changed_Credit_Limit       97909 non-null   float64
 13  Num_Credit_Inquiries       980

In [2209]:
for col in ["Num_Bank_Accounts", "Num_Credit_Card"]:
    df[col] = df[col].astype(str).str.replace(r"[^0-9]", "", regex=True)
    df[col] = pd.to_numeric(df[col])

In [2210]:
df["Num_Bank_Accounts"].isna().sum()

np.int64(0)

In [2211]:
df["Num_Credit_Card"].isna().sum()


np.int64(0)

In [2212]:
df["Interest_Rate"].isna().sum()

np.int64(0)

In [2213]:
df["Num_of_Loan"].isna().sum()

np.int64(0)

In [2214]:
df["Num_Bank_Accounts"].value_counts()

,count
Num_Bank_Accounts,
6,13001
7,12823
8,12765
4,12186
5,12118
...,...
1091,1
1123,1
1657,1


In [2215]:
for col in ["Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate", "Num_of_Loan"]:
    df[col] = df.groupby("Customer_ID")[col].transform(lambda x: x.mode()[0] if not x.mode().empty else x.median())

In [2216]:
df.loc[df["Num_Bank_Accounts"] == 0, "Num_Credit_Card"] = 0

In [2217]:
df["Num_Bank_Accounts"].value_counts()

,count
Num_Bank_Accounts,
6,13184
7,12976
8,12936
4,12392
5,12272
3,12096
9,5512
10,5328
1,4568


In [2218]:
df["Num_Credit_Card"].value_counts()

,count
Num_Credit_Card,
5,17928
6,16488
7,16400
4,13488
3,12736
8,5088
10,4960
9,4736
0,4400


In [2219]:
df["Interest_Rate"].value_counts()

,count
Interest_Rate,
8,5104
5,5096
6,4832
12,4648
10,4616
7,4584
9,4576
11,4512
18,4192


In [2220]:
df["Num_of_Loan"].value_counts()

/usr/local/lib/python3.12/dist-packages/IPython/core/displayhook.py:275: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


,count
Num_of_Loan,
3,15752
2,15712
4,15456
0,11408
1,11128
6,8144
7,7680
5,7528
9,3856


In [2221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Customer_ID                100000 non-null  object 
 1   Age                        100000 non-null  int64  
 2   Occupation                 100000 non-null  int64  
 3   Annual_Income              100000 non-null  float64
 4   Monthly_Inhand_Salary      100000 non-null  float64
 5   Num_Bank_Accounts          100000 non-null  int64  
 6   Num_Credit_Card            100000 non-null  int64  
 7   Interest_Rate              100000 non-null  int64  
 8   Num_of_Loan                100000 non-null  int64  
 9   Type_of_Loan               88592 non-null   object 
 10  Delay_from_due_date        100000 non-null  int64  
 11  Num_of_Delayed_Payment     92998 non-null   object 
 12  Changed_Credit_Limit       97909 non-null   float64
 13  Num_Credit_Inquiries       980

In [2222]:
df["Type_of_Loan"].isna().sum()

np.int64(11408)

In [2223]:
df["Type_of_Loan"] = df["Type_of_Loan"].fillna("Not Specified")

In [2224]:
def clean_loans(x):
    x = str(x).replace("and ", "")
    x = x.strip().strip(",")
    loans = [loan.strip() for loan in x.split(",") if loan.strip()]
    loans = [loan for loan in loans if loan.lower() != "nan"]
    return loans

df["Type_of_Loan"] = df["Type_of_Loan"].apply(clean_loans)


In [2225]:
df["Type_of_Loan"]

,Type_of_Loan
0,"[Auto Loan, Credit-Builder Loan, Personal Loan..."
1,"[Auto Loan, Credit-Builder Loan, Personal Loan..."
2,"[Auto Loan, Credit-Builder Loan, Personal Loan..."
3,"[Auto Loan, Credit-Builder Loan, Personal Loan..."
4,"[Auto Loan, Credit-Builder Loan, Personal Loan..."
...,...
99995,"[Auto Loan, Student Loan]"
99996,"[Auto Loan, Student Loan]"
99997,"[Auto Loan, Student Loan]"
99998,"[Auto Loan, Student Loan]"


In [2226]:
mlb = MultiLabelBinarizer()
loan_dummies = pd.DataFrame(
    mlb.fit_transform(df["Type_of_Loan"]),
    columns=[f"Loan_{c}" for c in mlb.classes_],
    index=df.index
    )

df = df.join(loan_dummies)
df = df.drop(columns=["Type_of_Loan"])


In [2229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Customer_ID                   100000 non-null  object 
 1   Age                           100000 non-null  int64  
 2   Occupation                    100000 non-null  int64  
 3   Annual_Income                 100000 non-null  float64
 4   Monthly_Inhand_Salary         100000 non-null  float64
 5   Num_Bank_Accounts             100000 non-null  int64  
 6   Num_Credit_Card               100000 non-null  int64  
 7   Interest_Rate                 100000 non-null  int64  
 8   Num_of_Loan                   100000 non-null  int64  
 9   Delay_from_due_date           100000 non-null  int64  
 10  Num_of_Delayed_Payment        92998 non-null   float64
 11  Changed_Credit_Limit          97909 non-null   float64
 12  Num_Credit_Inquiries          98035 non-null 

In [2241]:
df["Delay_from_due_date"].isna().sum()

np.int64(0)

In [2244]:
df["Delay_from_due_date"][1]

np.int64(-1)

In [2246]:
df[df["Delay_from_due_date"] < 0] = 0

In [2240]:
df["Num_of_Delayed_Payment"].isna().sum()

np.int64(7002)

In [2242]:
df["Num_of_Delayed_Payment"].fillna(df["Num_of_Delayed_Payment"].median(), inplace=True)

In [2243]:
df["Num_of_Delayed_Payment"].isna().sum()

np.int64(0)

In [2249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Customer_ID                   100000 non-null  object 
 1   Age                           100000 non-null  int64  
 2   Occupation                    100000 non-null  int64  
 3   Annual_Income                 100000 non-null  float64
 4   Monthly_Inhand_Salary         100000 non-null  float64
 5   Num_Bank_Accounts             100000 non-null  int64  
 6   Num_Credit_Card               100000 non-null  int64  
 7   Interest_Rate                 100000 non-null  int64  
 8   Num_of_Loan                   100000 non-null  int64  
 9   Delay_from_due_date           100000 non-null  int64  
 10  Num_of_Delayed_Payment        100000 non-null  float64
 11  Changed_Credit_Limit          97916 non-null   float64
 12  Num_Credit_Inquiries          98049 non-null 